# Email process
## Ver 2.0
### 运行程序前将原始csv文件保存在当前目录的Email子目录，合并多个原始的csv格式email文件，删除空列，并按照原始邮件和回复邮件分别保存在不同Questions_data.csv文件和replys_data.csv中，提取对分类有帮助的Subject和Body列合并为Content列，保存在newdata.csv中。

In [1]:
import os
import pandas as pd

In [2]:
base_dir = "./email"
df = pd.DataFrame()
for item in os.listdir(base_dir):
    f = open(os.path.join(base_dir, item), encoding="utf-8")
    dftemp=pd.read_csv(f)

    df=pd.concat([df,dftemp])
#df.reset_index(inplace = True) 
#df.describe
#删除空列
df.drop(columns=['BCC: (Name)', 'BCC: (Address)','BCC: (Type)','Billing Information','Mileage'], inplace=True)
#删除Subject为空的行（只有1行不影响结果，第525行）
df.dropna(subset=['Subject'],inplace = True)
# 消除Subject首尾空格
df['Subject'] = df['Subject'].str.strip()
# 去掉多余的空格,步骤如下：
#   1.将字符串以空格拆分为列表，如"you are     man"拆分为["you","are","man"]
df['Subject']=df['Subject'].str.split()
#   2.将列表用空格连接，多余的空格被去掉
df['Subject']=df['Subject'].apply(' '.join)
# 同上
df['Body']=df['Body'].str.split()
df['Body']=df['Body'].apply(' '.join)
# 去掉多余的换行符，步骤如下：
#   1.将字符串以换行符(\n)拆分为列表，如"you\nare\n\n\nman"拆分为["you",,"are",,,"man"]
df['Body']=df['Body'].str.split('\n')
#   2.将列表中的空项删除
df['Body']=df['Body'].apply(lambda x:[t for t in x if len(t)>0])
#   3，将列表用换行符(\n)连接，多余的换行符被去掉
df['Body']=df['Body'].apply('\n'.join)

#print(df.info())
# 取出回复的邮件（邮件题目以RE:或FW:开始的邮件）
replysdf=pd.concat([df[df[df['Subject'].notnull()]['Subject'].str.startswith('RE:')]['Subject'],df[df[df['Subject'].notnull()]['Subject'].str.startswith('FW:')]['Subject']])
replysdf.to_csv(os.path.join(base_dir,"replys_data.csv"))
# 取出原始的提问邮件
questionsdf=df[~df.index.isin(replysdf.index)]
questionsdf.reset_index(inplace = True) 
questionsdf.to_csv(os.path.join(base_dir,"Questions_data.csv"))
print(questionsdf.info())
print(questionsdf)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            580 non-null    int64 
 1   Subject          580 non-null    object
 2   Body             580 non-null    object
 3   From: (Name)     580 non-null    object
 4   From: (Address)  580 non-null    object
 5   From: (Type)     580 non-null    object
 6   To: (Name)       564 non-null    object
 7   To: (Address)    564 non-null    object
 8   To: (Type)       564 non-null    object
 9   CC: (Name)       90 non-null     object
 10  CC: (Address)    90 non-null     object
 11  CC: (Type)       90 non-null     object
 12  Categories       253 non-null    object
 13  Importance       580 non-null    object
 14  Sensitivity      580 non-null    object
dtypes: int64(1), object(14)
memory usage: 68.1+ KB
None
     index                                            Subject  \
0       

In [ ]:
# 查看其它列数据
cols=['From: (Name)','To: (Name)','CC: (Name)','Categories','Importance','Sensitivity']
for col in cols:
    group = questionsdf.groupby(col) # 按照'CC: (Name)'列的值来分组，创建一个groupby对象
    print('------ ',col, ' -----')
    for key, df in group:
        print(key,len(df))

### 暂时未发现数据对于分类有帮助，可暂时不考虑，以下仅处理Subject和Body，合并为Content列

In [ ]:
# 合并'Subject','Body'列，生成newdata.csv文件
newdf=questionsdf[['Subject','Body']]
newdf=newdf.assign(Content=newdf['Subject'] + '\n' + newdf['Body'])
newdf.info()
newdf.to_csv(os.path.join(base_dir,"newdata.csv"))